To score flood predictions you must have flood labels available of course. Those should be added to your datafolder in a similar structure to the example in `data/inference_sample`.

In [ ]:
import cv2
import numpy as np
from pathlib import Path
from flood_mapping.utils import create_path_df

data_folder = Path("data/inference_sample")
path_df = create_path_df(data_folder, add_flood_label_paths=True)

It is calculated with IOU score, Intersect Over Union, which in short gives penalties for both types of errors. If you predict a flood at a pixel where there is none, the union gets larger. If you don't predict a flood at a pixel where there is one, the intersect gets smaller. Both result in a lower score compared to the score of 1 if you predict all floods correctly and none incorrectly (intersect=union, IOU=1).

In [ ]:
def calc_iou(label, pred):
    intersection = np.logical_and(label, pred)
    union = np.logical_or(label, pred)
    iou_score = np.sum(intersection) / np.sum(union)
    return(iou_score)


In [ ]:
iou_scores_never_flood = list()
iou_scores = list()
iou_scores_postprocessed = list()
iou_scores_posthigh = list()
never_flood_pred = np.zeros((256, 256))

for pred_path, label_path in zip(path_df['pred_path'], path_df['flood_label_path']):
    flood_label = cv2.imread(str(label_path), 0) / 255
    flood_pred = cv2.imread(str(pred_path), 0) / 255

    iou_score = calc_iou(flood_label, flood_pred)
    iou_scores.append(iou_score)


iou_scores = np.array(iou_scores)

It is unclear whether the challenge where this nvidia model was taken from counted predicting no floods for an image with no floods as a score of 1 for that image and took the average of all individual IOU scores to get the challenge rating, or that the IOU score is calculated pasting all images together to one big image, ignoring the non flood images. The latter is most commonly how we find it described in literature. Both scores are calculated below:

In [ ]:
print(f'{np.nanmean(iou_scores):0.3} - IOU Score ignoring non flood images')
print(f'{np.nan_to_num(iou_scores, nan=1).mean():0.3} - IOU Score where correct non flood images count as 1')

We can visualise the scores in a bar chart to check 

In [ ]:
import plotly.express as px

px.histogram(np.nan_to_num(iou_scores, nan=1), title='IOU score distribution').show()